In [ ]:
import os
import cv2
from pathlib import Path
import open3d as o3d
import numpy as np
import rootutils

rootutils.setup_root(
    os.path.abspath(''), indicator=['.git', 'pyproject.toml'], pythonpath=True
)

from src.stereo_inference import StereoInferenceOnnx

from src.stereo_calibrate import (
    read_stereo_calibration,
    read_camera_intrinsics,
)
from src.utils import (
    StereoImages,
    visualize_disparity,
    postprocess_disparity,
    create_point_cloud,
    disparity_to_depth_map,
)

In [ ]:
# Load stereo calibration parameters
stereo_map_left, stereo_map_right = read_stereo_calibration(
    '../camera_configs/stereo_calibration.xml'
)
# Read camera intrinsics
K, distance_between_cameras = read_camera_intrinsics(
    '../camera_configs/left_camera_intrinsics.xml'
)
# Load stereo model
stereo_model = StereoInferenceOnnx('../models/fs_800_640.onnx')
# Initialize stereo image processor
processor = StereoImages(stereo_map_left, stereo_map_right)

In [ ]:
# gather images of left and right
image_left_dir = Path('../data/left_anomaly_2/')
image_right_dir = Path('../data/right_anomaly_2/')
image_left_paths = sorted(list(image_left_dir.glob('*.png')))
image_right_paths = sorted(list(image_right_dir.glob('*.png')))

# iterate through image pairs
for img_left_path, img_right_path in zip(image_left_paths, image_right_paths):
    proc_result = processor.process(
        img_left_path,
        img_right_path,
        stereo_model.input_width,
        stereo_model.input_height
    )

    left_tensor, right_tensor = proc_result['tensors']
    scale = proc_result['scale']
    left_rectified = proc_result['rectified_images'][0]

    # Stereo inference
    disparity_map = stereo_model(left_tensor, right_tensor)

    # squeeze form 4D to 2D
    disparity_map = disparity_map.squeeze()
    # Visualize disparity map
    disparity_map_viz = visualize_disparity(disparity_map, mask=None)
    # Postprocess disparity map
    disparity_map_proc = postprocess_disparity(disparity_map, mask=None)
    # depth map
    depth_map = disparity_to_depth_map(disparity_map, K, distance_between_cameras, scale)
    # post process depth map
    depth_map_proc = postprocess_disparity(depth_map, mask=None)

    # save disparity map as numpy file
    np.save(f'../output/disp/disparity_map_{img_left_path.stem}.npy', depth_map_proc)
    # save disparity map
    cv2.imwrite(f'../output/disp/disparity_map_{img_left_path.stem}.png', cv2.cvtColor(disparity_map_viz, cv2.COLOR_RGB2BGR))
    # Create point cloud
    pcd = create_point_cloud(
        disparity_map_proc, left_rectified, K, distance_between_cameras, scale=scale
    )

    # Save point cloud to PLY file
    output_path = Path('../output/pcd/')
    output_path.mkdir(parents=True, exist_ok=True)
    o3d.io.write_point_cloud(str(output_path / (img_left_path.stem + '.ply')), pcd)